<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/misc/chpl_rest_api_sierra_bulk_item_level_holds_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="250"/>

# REST API: Bulk Item-level Holds Report

Given a spreadsheet of barcodes, produce an output consisting of
* `id`: item record number,
* `location`: current item location code and name,
* `status`: item status code and description,
* `barcode`: item barcode,
* `bibIds`: list of bib record numbers associated with item record number, 
* `volumes`: list of volume record numbers associated with item record number,
* `holdCount`: count of holds on the item

---

## 1. Enter REST-API Credentials

In order to be authorized to use the REST API, you must enter valid credientials.

 ↴ **Click on the play button below**, then when prompted, enter in the **`auth_string`** as your password:

In [ ]:
#@title
import re
import time

# this is the helper script that contains some needed functions
!wget --quiet https://raw.githubusercontent.com/cincinnatilibrary/collection-analysis/8352bf7d9debf027e760e426c6d54a1df3aee77c/misc/chpl_helper.py --output-document=chpl_helper.py
!pip install -U rich > /dev/null
time.sleep(1)

# import the helper functions we just downloaded ...
import chpl_helper
from getpass import getpass, getuser
from google.colab import files
import pandas as pd
import requests
import json
from io import StringIO
from rich.progress import Progress, SpinnerColumn, TextColumn, track
from rich import print
import logging

# create log file
ts = pd.Timestamp('now').isoformat()
logging.basicConfig(
    filename="bulk_holds_check_log.txt", 
    format="%(asctime)s\t%(levelname)s\t%(message)s", 
    level=logging.DEBUG
)
logging.info('script started: {}'.format(ts))

# point this to the base URL for the Sierra REST API
base_url = 'https://classic.cincinnatilibrary.org:443/iii/sierra-api/v6/'

# client_key = getpass('Enter the client KEY value: ')
# client_secret = getpass('Enter the client SECRET value: ')
auth_string = getpass('Enter the auth_string value: ')

# test our credentials

try:
  headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)
  r = requests.get(base_url + 'info/token', headers=headers, verify=True)
  print('access token expires in: {} seconds'.format(r.json()['expiresIn']))
  logging.info('keyId: {}'.format(r.json()['keyId']))
  logging.info('expiresIn: {}'.format(r.json()['expiresIn']))
except:
  logging.error('requst info/token status:{} \tcontent:'.format(r.status_code, r.content))

# grab the number at the end of the url, which _should_ be the item record num
re_item_num = re.compile(r"^.*\/([0-9]{1,})$")

## 2. Upload Input File

This file will be processed to placed the holds.

 ↴ **Click on the play button below**, then when prompted, upload the input file:

In [ ]:
#@title
input_file = files.upload()

# convert the input file to a Dataframe ... 
df = pd.read_excel( 
    [key for key in input_file.keys()][0],
    usecols=[0,1],
    converters={
        0: str,
        1: str
    }
)
# print(df.head(5))
# print('...')
print('total rows in input: ', df.shape[0])

# construct the item query we'll use to convert barcodes to item record numbers 
# (API endpoint URLs)
operands = [row['barcode'] for i, row in df.iterrows()]
item_query = {
  "queries": [
    {
      "target": {
        "record": {
          "type": "item"
        },
        "field": {
          "tag": "b"
        }
      },
      "expr": [
        {
          "op": "in",
          "operands": operands
        }
      ]
    }
  ]
}

# print(json.dumps(item_query))

headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)

try:
  text_column = TextColumn("Searching")
  spinner_column = SpinnerColumn(finished_text='Complete!')
  with Progress(text_column, spinner_column, expand=False) as progress:
  #Progress(transient=True) as progress:
    task = progress.add_task("Searching ...")

    r = requests.post(
      base_url + 'items/query?offset=0&limit={}'.format(len(operands)), 
      headers=headers,
      data=json.dumps(item_query),
      verify=True
    )
    progress.update(task, completed=1)

  # print("response json: ", r.json())
  print('...')
  msg = 'total rows of items returned from search: {}'.format(r.json()['total'])
  logging.info(msg)
  print(msg)

except:
  logging.error('items/query failed')
  print('error?')

print('Building Report...')

record_nums = []

for i, entry in enumerate(r.json()['entries']):
  # print(i, entry, end='', sep=' ')
  try:
    record_nums.append(
        # extract the item record number from the link
        re_item_num.match(entry['link'])[1]
    )
  except:
    pass

entries = []

for i in range(0, len(record_nums), 100):
  # id,location,status,barcode,bibIds,volumes,holdCount

  # 3154454,3021010

  # https://classic.cincinnatilibrary.org:443/iii/sierra-api/v6/items/?limit=2&id=3154454%2C3021010&fields=id%2Clocation%2Cstatus%2Cbarcode%2CbibIds%2Cvolumes%2CholdCount
  try:
    r = requests.get(
        url = base_url + 'items/',
        headers=headers,
        params={
            'limit': 100,
            'offset': 0,
            'id': ','.join([str(num) for num in record_nums][i:i+100]),
            'fields': 'id,location,status,barcode,bibIds,volumes,holdCount',
        },
        verify=True
    )

  except:
    print('error')

  entries.extend(r.json()['entries'])
  
  # if i > 200:
  #   break

pd.DataFrame(entries).to_csv('output.csv')

## 3. Retrieve the file

 ↩ **Click on the "Files" button (image.png) to the left, right click the file `output.txt` and then click `Download`**